In [ ]:
import ase.io as ase_io
import numpy as np

In [ ]:
def get_rmse(a, b, perc=False):
    rmse = np.sqrt(np.mean(np.square(a - b)))
    if perc:
        return 100 * rmse / b.std(ddof=1)
    return rmse

In [ ]:
mols = ["aspirin", "ethanol", "malonaldehyde", "naphthalene", "salicylic", "toluene"]

In [ ]:
frames = {}
for mol in mols:
    frames[mol] = ase_io.read(f"../../data/rMD17/{mol}.xyz", ":")

In [ ]:
energy_err = {}
for mol in mols:

    gomace = np.asarray(
        [
            [
                x.info["gomace_L0_energy"],
                x.info["gomace_energy"],
                x.info["gomace_L2_energy"],
            ]
            for x in frames[mol]
        ]
    )

    gomace -= gomace[0]
    gomace = gomace[1:]
    # gomace = gomace.reshape(3, -1)

    rmd17 = np.array([[x.info["rMD17_energy"] for _ in range(3)] for x in frames[mol]])
    rmd17 -= rmd17[0]
    rmd17 = rmd17[1:]
    # rmd17 = rmd17.reshape(3, -1)

    energy_err[mol] = {}
    energy_err[mol]["rmd17"] = (
        1e3
        # * np.array([get_rmse(x, y) for x, y in zip(gomace, rmd17)])
        * np.sqrt(np.mean((gomace - rmd17) ** 2, axis=0))
        / len(frames[mol][0])
    )

In [ ]:
forces_err = {}
for mol in mols:

    gomace = np.asarray(
        [
            [
                x.arrays["gomace_L0_forces"],
                x.arrays["gomace_forces"],
                x.arrays["gomace_L2_forces"],
            ]
            for x in frames[mol]
        ]
    )

    rmd17 = np.array(
        [[x.arrays["rMD17_forces"] for _ in range(3)] for x in frames[mol]]
    )

    forces_err[mol] = {}
    forces_err[mol]["rmd17"] = 1e3 * np.sqrt(
        np.mean((gomace - rmd17) ** 2, axis=(0, 2, 3))
    )

In [ ]:
for mol in mols:
    emd17 = [f"{e:.1f}" for e in energy_err[mol]["rmd17"]]
    fmd17 = [f"{1e-3*e:.2f}" for e in forces_err[mol]["rmd17"]]
    print(mol + " &  " + " & ".join(emd17) + " & & " + " & ".join(fmd17) + "\\\\")